# **Drug Trafficking (2008 - 2011) Study Case**
<img src="./images/sof_insignia.png" width="350px">

In [1]:
import pandas as pd 
import numpy as np 
import json
import pymysql 
import sqlite3 
import os 
import boto3 
import warnings 
from sqlalchemy import create_engine
import gc 
import seaborn 
import matplotlib.pyplot as plt 

from typing import List, Dict 

warnings.filterwarnings('ignore')
plt.style.use('ggplot')

# **Helper Function**
## **To MySQL** 

In [3]:
def to_mysql(backend_engine: 'PyMsql', dir_path: str ):
    """
    Description
    -----------
        - Helper function to convert the CDE dataset into MySQL DB for further query

    Parameter
    ---------
        - backend_engine: a valid SQLAlchemy backend engine 
        - dir_path: a valid CDE path 

    Return
    ------
        - None 
    """

    if not os.path.exists(dir_path):
        raise FileNotFoundError(f"[ ERROR ] Unable to find the following CDE dataset path {dir_path}")

    try:
        
        for csv_file in os.listdir(dir_path):
            if "csv" in csv_file:
                tbl_name: str = csv_file.replace(".csv", "").strip()
                pd.read_csv(os.path.join(dir_path, csv_file) ).to_sql(tbl_name, con=backend_engine, if_exists='append' )

    except ConnectionError as e: 
        
        raise ConnectionError("[ ERROR ] The program has encountered the following error message. Please check your SQL connection !!!.") from e 

## **MySQL Cursor** 

In [2]:
con: 'MySQL' = pymysql.connect(user="root", passwd="admin123", host="127.0.0.1")
cursor = con.cursor() 

secrets: 'SecretsManager' = boto3.client("secretsmanager")
sql_arg: str = json.loads(secrets.get_secret_value(SecretId="afghan_project").get("SecretString")).get("sql_args")

## **Populate TX DB with Crimes Incidents between `2008 - 2011`**

In [5]:
engine: 'pymysql' = create_engine("%s%s" % (sql_arg, "TX_2008") )
to_mysql(backend_engine=engine, dir_path="/tmp/TX/2008/")

engine: 'pymysql' = create_engine("%s%s" % (sql_arg, "TX_2009") )
to_mysql(backend_engine=engine, dir_path="/tmp/TX/2009/")

engine: 'pymysql' = create_engine("%s%s" % (sql_arg, "TX_2010") )
to_mysql(backend_engine=engine, dir_path="/tmp/TX/2010/")

engine: 'pymysql' = create_engine("%s%s" % (sql_arg, "TX_2011") )
to_mysql(backend_engine=engine, dir_path="/tmp/TX/2011/")

## **Populate Arizona DB with Crimes Incidents between `2008 - 2011`**

In [9]:
engine: 'pymysql' = create_engine("%s%s" % (sql_arg, "AZ_2008") )
to_mysql(backend_engine=engine, dir_path="/tmp/AZ/2008/")

engine: 'pymysql' = create_engine("%s%s" % (sql_arg, "AZ_2009") )
to_mysql(backend_engine=engine, dir_path="/tmp/AZ/2009/")

engine: 'pymysql' = create_engine("%s%s" % (sql_arg, "AZ_2010") )
to_mysql(backend_engine=engine, dir_path="/tmp/AZ/2010/")

engine: 'pymysql' = create_engine("%s%s" % (sql_arg, "AZ_2011") )
to_mysql(backend_engine=engine, dir_path="/tmp/AZ/2011/")

## **Index Table for All the Crimes in Az and Tx between `2008 - 2011`**
<img src="images/heroin_routes.png" width="1100px">

## **Drufg Trafficking Cases in Arizona** 

In [4]:
cde_events: Dict = dict(pd.read_sql("SHOW DATABASES", con=con)['Database'].apply(lambda row: row if 'AZ' in row or 'TX' in row else np.nan ).dropna().apply(lambda row:  (row, "%s%s" % (sql_arg, row) ) ).tolist() )
az2008 = create_engine( cde_events.get("AZ_2008") )
az2009 = create_engine( cde_events.get("AZ_2009") )
az2010 = create_engine( cde_events.get("AZ_2010") )
az2011 = create_engine( cde_events.get("AZ_2011") )

In [ ]:
# preps, reporting_agency, incident_location, quantity, race/ethnicity
query: str = """ 
/* 
    Description: Query for drug related charges in Arizona between 2008 - 2011 
*/

SELECT 
    DISTINCT 
    /*Incident Detail*/
    ni.incident_date, 
    CAST( IFNULL(ni.incident_hour,0) AS DECIMAL) AS `incident_hour`, 
    IFNULL(ni.submission_date, "n/a") AS `submission_date`,

    /*Offender Detail*/
    IFNULL(noff.age_num, 0) AS `age_num`,
    IFNULL(noff.sex_code, "Unknown") AS `sex_code`, 
    IFNULL(noff.age_range_low_num, 0) AS `age_range_low_num`, 
    IFNULL(noff.age_range_high_num, 0) AS `age_range_high_num`,
    narr.arrest_num,
    narr.arrest_date, 
    narr.age_num, 
    narr.sex_code,
    IFNULL(rr.race_desc, "Unknown") AS `race_desc`, 
    IFNULL(rr.notes , "Unknown") AS `race_notes`, 
    IFNULL(nwt.weapon_name, "Unknown") AS `weapon_name`, 
    IFNULL(nsd.est_drug_qty,0) AS `est_drug_qty`,
    IFNULL(nsdt.suspected_drug_name, "Unknown") AS `suspected_drug_name`,
    
    /*Crime Category*/
    ncat.criminal_act_name,
    
    /*Reporting Agency*/
    cdea.agency_name,
    cdea.short_name,
    cdea.city_name,
    cdea.state_abbr,
    cdea.current_year, 
    cdea.months_reported,
    cdea.nibrs_months_reported,
    cdea.past_10_years_reported
    
    
FROM nibrs_offense no 
JOIN nibrs_criminal_act nca ON nca.offense_id = no.offense_id
JOIN nibrs_criminal_act_type ncat ON ncat.criminal_act_id = nca.criminal_act_id
JOIN nibrs_incident ni ON ni.incident_id = no.incident_id
JOIN nibrs_offender noff ON noff.incident_id = ni.incident_id
JOIN nibrs_arrestee narr ON narr.incident_id = ni.incident_id AND narr.offense_type_id = no.offense_type_id
JOIN ref_race rr ON rr.race_id = narr.race_id 
JOIN nibrs_arrestee_weapon naw ON naw.arrestee_id = narr.arrestee_id
JOIN nibrs_weapon_type nwt ON nwt.weapon_id = naw.weapon_id 
JOIN cde_agencies cdea ON cdea.agency_id = ni.agency_id 
JOIN nibrs_property np ON np.incident_id = ni.incident_id 
JOIN nibrs_suspected_drug nsd ON nsd.property_id = np.property_id 
JOIN nibrs_suspected_drug_type nsdt ON nsdt.suspected_drug_type_id = nsd.suspected_drug_type_id

LIMIT 5
""" 

az_crimes: 'DataFrame' = pd.read_sql(query, con=az2008)
az_crimes.head() 

## **Drufg Trafficking Cases in Texas** 

# **References** 
1. https://www.justice.gov/archive/ndic/pubs44/44849/44849p.pdf